# Data Aggregation and Group Operations

In [1]:
import numpy as np 
import pandas as pd

## How to think about Group operations

In [2]:
frame = pd.DataFrame({
    "key": ['a','a', None, 'b','b','a', None], 
    "key2": pd.Series([1,2,1,2,1,None,1], dtype="Int64"),
    "data1": np.random.standard_normal(7),
    "data2": np.random.standard_normal(7),
})

frame

,key,key2,data1,data2
0,a,1,-0.474720,1.702165
1,a,2,1.856472,-1.377434
2,None,1,-0.063756,0.285598
3,b,2,-0.141733,0.400452
4,b,1,1.356056,-2.207610
5,a,<NA>,0.012307,-0.025207
6,None,1,0.400062,0.884155


In [3]:
grouped = frame['data1'].groupby(frame['key'])
grouped.mean()

key
a    0.464686
b    0.607161
Name: data1, dtype: float64

In [4]:
means = frame['data1'].groupby([frame['key'], frame['key2']]).mean()
means

key  key2
a    1      -0.474720
     2       1.856472
b    1       1.356056
     2      -0.141733
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,1,2
key,,
a,-0.474720,1.856472
b,1.356056,-0.141733


In [6]:
states = np.array(['oh', 'ca', 'ca', 'oh', 'oh', 'ca', 'oh'])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

# Same as concatenating two new columns and then grouping by their names
frame['data1'].groupby([states, years]).mean()

ca  2005    0.934389
    2006   -0.063756
oh  2005   -0.308226
    2006    0.878059
Name: data1, dtype: float64

In [7]:
frame.groupby('key').mean(),frame.groupby('key2').mean(),frame.groupby(['key', 'key2']).mean()

C:\Users\William Costa\AppData\Local\Temp\ipykernel_8988\2336427859.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  frame.groupby('key').mean(),frame.groupby('key2').mean(),frame.groupby(['key', 'key2']).mean()


(     key2     data1     data2
 key                          
 a     1.5  0.464686  0.099841
 b     1.5  0.607161 -0.903579,
          data1     data2
 key2                    
 1     0.304411  0.166077
 2     0.857369 -0.488491,
              data1     data2
 key key2                    
 a   1    -0.474720  1.702165
     2     1.856472 -1.377434
 b   1     1.356056 -2.207610
     2    -0.141733  0.400452)

In [8]:
frame.groupby(['key', 'key2']).size(), frame.groupby(['key', 'key2'],dropna=False).size()

(key  key2
 a    1       1
      2       1
 b    1       1
      2       1
 dtype: int64,
 key  key2
 a    1       1
      2       1
      <NA>    1
 b    1       1
      2       1
 NaN  1       2
 dtype: int64)

In [9]:
frame.groupby('key').count()

,key2,data1,data2
key,,,
a,2,3,3
b,2,2,2


## Iterating over Groups

In [10]:
for name, group in frame.groupby('key'):
    print(name)
    print(group)

a
  key  key2     data1     data2
0   a     1 -0.474720  1.702165
1   a     2  1.856472 -1.377434
5   a  <NA>  0.012307 -0.025207
b
  key  key2     data1     data2
3   b     2 -0.141733  0.400452
4   b     1  1.356056 -2.207610


In [11]:
for (key1, key2), group in frame.groupby(['key', 'key2']):
    print((key1,key2))
    print(group)

('a', 1)
  key  key2    data1     data2
0   a     1 -0.47472  1.702165
('a', 2)
  key  key2     data1     data2
1   a     2  1.856472 -1.377434
('b', 1)
  key  key2     data1    data2
4   b     1  1.356056 -2.20761
('b', 2)
  key  key2     data1     data2
3   b     2 -0.141733  0.400452


In [12]:
pieces = {name: group for name, group in frame.groupby('key')}
pieces

{'a':   key  key2     data1     data2
 0   a     1 -0.474720  1.702165
 1   a     2  1.856472 -1.377434
 5   a  <NA>  0.012307 -0.025207,
 'b':   key  key2     data1     data2
 3   b     2 -0.141733  0.400452
 4   b     1  1.356056 -2.207610}

In [13]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

a
0   -0.474720
1    1.856472
5    0.012307
Name: data1, dtype: float64
b
3   -0.141733
4    1.356056
Name: data1, dtype: float64


## Grouping with Dictionaries and Series

In [14]:
people = pd.DataFrame(np.random.standard_normal((5,5)), columns=list('abcde'), index=['Joe', 'Steve', 'Wanda', 'Jill', 'Trey'])

people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.396318,-0.573523,-0.283418,-1.649333,-0.328246
Steve,0.754424,0.562681,0.430825,0.009879,0.660069
Wanda,-0.279720,NaN,NaN,0.348775,-1.636688
Jill,2.273280,-1.130728,0.212475,0.120659,-0.732249
Trey,-1.656412,-0.724789,0.237338,2.560531,-0.686809


In [15]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis='columns')

by_column.sum()

,blue,red
Joe,-1.932751,-0.505451
Steve,0.440704,1.977175
Wanda,0.348775,-1.916409
Jill,0.333134,0.410302
Trey,2.797869,-3.068009


In [16]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [17]:
people.groupby(map_series, axis='columns').count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


## Grouping with Functions

In [18]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.396318,-0.573523,-0.283418,-1.649333,-0.328246
4,0.616869,-1.855517,0.449813,2.681190,-1.419058
5,0.474704,0.562681,0.430825,0.358653,-0.976619


In [19]:
key_list = ['one', 'two', 'three', 'four', 'five']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one    0.396318 -0.573523 -0.283418 -1.649333 -0.328246
4 five  -1.656412 -0.724789  0.237338  2.560531 -0.686809
  four   2.273280 -1.130728  0.212475  0.120659 -0.732249
5 three -0.279720       NaN       NaN  0.348775 -1.636688
  two    0.754424  0.562681  0.430825  0.009879  0.660069

## Grouping by Index Level

In [20]:
columns = pd.MultiIndex.from_arrays([
    ['us','us','us','jp','jp'],
    [1,3,5,1,3]],
    names=['city', 'tenor']
)
hierarchical_frame = pd.DataFrame(np.random.standard_normal((4,5)), columns = columns)
hierarchical_frame

city         us                            jp          
tenor         1         3         5         1         3
0      1.576433  0.070524  1.391640  0.845870  2.252414
1      0.207661 -1.742044 -0.852452 -1.754730  0.771036
2      0.663477  1.362904  0.838972  0.413350  0.323356
3      0.478514  0.589996 -0.152284  0.866715  0.441258

In [21]:
hierarchical_frame.groupby(level='city', axis='columns').count()

city,jp,us
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [22]:
grouped = frame.groupby('key')
grouped['data1'].min(),grouped['data1'].nsmallest(2)

(key
 a   -0.474720
 b   -0.141733
 Name: data1, dtype: float64,
 key   
 a    0   -0.474720
      5    0.012307
 b    3   -0.141733
      4    1.356056
 Name: data1, dtype: float64)

In [23]:
aggregation_function = lambda array: array.max() - array.min()

grouped.agg(aggregation_function)

,key2,data1,data2
key,,,
a,1,2.331191,3.079599
b,1,1.497789,2.608062


In [24]:
grouped.describe()

key2                                           data1            ...  \
    count mean       std  min   25%  50%   75%  max count      mean  ...   
key                                                                  ...   
a     2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.464686  ...   
b     2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.607161  ...   

                        data2                                          \
          75%       max count      mean       std       min       25%   
key                                                                     
a    0.934389  1.856472   3.0  0.099841  1.543603 -1.377434 -0.701321   
b    0.981609  1.356056   2.0 -0.903579  1.844178 -2.207610 -1.555595   

                                   
          50%       75%       max  
key                                
a   -0.025207  0.838479  1.702165  
b   -0.903579 -0.251564  0.400452  

[2 rows x 24 columns]

## Column-Wise and Multiple Function Application

In [25]:
location = r"C:\Users\William Costa\Documents\repositories\data_science_basic_knowledge\resources\tips.csv"
tips = pd.read_csv(location)

tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [26]:
tips['tips_pct'] = tips['tip'] / tips['total_bill']

tips.head()

,total_bill,tip,smoker,day,time,size,tips_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [27]:
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tips_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [28]:
grouped_pct.agg(['mean', 'std', aggregation_function])

mean       std  <lambda_0>
day  smoker                                
Fri  No      0.151650  0.028123    0.067349
     Yes     0.174783  0.051293    0.159925
Sat  No      0.158048  0.039767    0.235193
     Yes     0.147906  0.061375    0.290095
Sun  No      0.160113  0.042347    0.193226
     Yes     0.187250  0.154134    0.644685
Thur No      0.160298  0.038774    0.193350
     Yes     0.163863  0.039389    0.151240

In [29]:
grouped_pct.agg(['mean', 'std', ("max-min", aggregation_function)])

mean       std   max-min
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [30]:
functions = ['count', 'mean', 'max']
result = grouped[['tips_pct', 'total_bill']].agg(functions)
result

tips_pct                     total_bill                  
               count      mean       max      count       mean    max
day  smoker                                                          
Fri  No            4  0.151650  0.187735          4  18.420000  22.75
     Yes          15  0.174783  0.263480         15  16.813333  40.17
Sat  No           45  0.158048  0.291990         45  19.661778  48.33
     Yes          42  0.147906  0.325733         42  21.276667  50.81
Sun  No           57  0.160113  0.252672         57  20.506667  48.17
     Yes          19  0.187250  0.710345         19  24.120000  45.35
Thur No           45  0.160298  0.266312         45  17.113111  41.19
     Yes          17  0.163863  0.241255         17  19.190588  43.11

In [31]:
functions = [('Average', 'mean'), ('Variance', np.var)]
result = grouped[['tips_pct', 'total_bill']].agg(functions)
result

tips_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [32]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [33]:
grouped.agg({'tips_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tips_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [34]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

C:\Users\William Costa\AppData\Local\Temp\ipykernel_8988\2496195394.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(['day', 'smoker'], as_index=False).mean()


,day,smoker,total_bill,tip,size,tips_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [36]:
def top_values(frame: pd.DataFrame, n: int=5, column : str = 'tips_pct'):
    return frame.sort_values(column, ascending=False)[:n]

top_values(tips, n=6)

,total_bill,tip,smoker,day,time,size,tips_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [38]:
tips.groupby('smoker').apply(top_values)

total_bill   tip smoker   day    time  size  tips_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [40]:
tips.groupby(['smoker', 'day']).apply(top_values, n=2, column='total_bill')

total_bill    tip smoker   day    time  size  tips_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999

In [41]:
result = tips.groupby('smoker')['tips_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [42]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [53]:
from pandas.core.groupby.generic import DataFrameGroupBy
def group_function(group: DataFrameGroupBy):
    return group.describe()

grouped.apply(group_function)

total_bill       tip  size  tips_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Supressing Group Keys

In [49]:
tips.groupby('smoker').apply(top_values), tips.groupby('smoker', group_keys=False).apply(top_values)

(            total_bill   tip smoker   day    time  size  tips_pct
 smoker                                                           
 No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
        149        7.51  2.00     No  Thur   Lunch     2  0.266312
        51        10.29  2.60     No   Sun  Dinner     2  0.252672
        185       20.69  5.00     No   Sun  Dinner     5  0.241663
        88        24.71  5.85     No  Thur   Lunch     2  0.236746
 Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
        178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
        67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
        183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
        109       14.31  4.00    Yes   Sat  Dinner     2  0.279525,
      total_bill   tip smoker   day    time  size  tips_pct
 232       11.61  3.39     No   Sat  Dinner     2  0.291990
 149        7.51  2.00     No  Thur   Lunch     2  0.266312
 51        10.2

### Quantile and Bucket Analysis

In [50]:
frame = pd.DataFrame({
    'data1': np.random.standard_normal(1000),
    'data2': np.random.standard_normal(1000),
})

frame.head()

,data1,data2
0,-0.271567,1.039852
1,0.521767,1.309981
2,-0.950492,0.548293
3,-0.160550,-0.700325
4,0.429819,1.203983


In [55]:
def get_stats(group: DataFrameGroupBy):
    return pd.DataFrame({
        'min': group.min(), 'max': group.max(),
        'count': group.count(), 'mean': group.mean(),
    })
 
quartiles = pd.cut(frame['data1'], 4)

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.101, -1.541] data1 -3.094805 -1.564727     66 -2.003406
                 data2 -3.148839  2.288541     66 -0.096421
(-1.541, 0.0123] data1 -1.538797  0.011150    458 -0.628015
                 data2 -3.034631  2.434952    458  0.012970
(0.0123, 1.566]  data1  0.012600  1.548807    418  0.671460
                 data2 -2.522121  3.019784    418  0.063232
(1.566, 3.119]   data1  1.580593  3.119309     58  2.027812
                 data2 -3.000555  2.417324     58  0.051964

In [56]:
grouped.agg(['min', 'max', 'count', 'mean'])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.101, -1.541] -3.094805 -1.564727    66 -2.003406 -3.148839  2.288541   
(-1.541, 0.0123] -1.538797  0.011150   458 -0.628015 -3.034631  2.434952   
(0.0123, 1.566]   0.012600  1.548807   418  0.671460 -2.522121  3.019784   
(1.566, 3.119]    1.580593  3.119309    58  2.027812 -3.000555  2.417324   

                                  
                 count      mean  
data1                             
(-3.101, -1.541]    66 -0.096421  
(-1.541, 0.0123]   458  0.012970  
(0.0123, 1.566]    418  0.063232  
(1.566, 3.119]      58  0.051964

### Example: Filling Missing Values with Group Specific Values

In [60]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s.fillna(s.mean())

0   -0.309431
1    0.435223
2   -0.309431
3   -0.918227
4   -0.309431
5   -0.445290
dtype: float64

In [62]:
states = ['ohio', 'ny','vermont', 'florida', 'oregon', 'nevada', 'california','idaho']
group_key = ['East', 'East','East', 'East', 'West', 'West', 'West','West']

data = pd.Series(np.random.standard_normal(8), index=states)
data

ohio         -0.504705
ny            0.774080
vermont       1.584709
florida       3.184256
oregon        0.740273
nevada        0.477904
california   -0.697979
idaho        -0.025083
dtype: float64

In [63]:
data[['vermont', 'nevada', 'idaho']] = np.nan
data

ohio         -0.504705
ny            0.774080
vermont            NaN
florida       3.184256
oregon        0.740273
nevada             NaN
california   -0.697979
idaho              NaN
dtype: float64

In [64]:
data.groupby(group_key).agg(['size', 'count', 'mean'])

,size,count,mean
East,4,3,1.151210
West,4,2,0.021147


In [65]:
def fill_mean(group: DataFrameGroupBy):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

C:\Users\William Costa\AppData\Local\Temp\ipykernel_8988\492282649.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


ohio         -0.504705
ny            0.774080
vermont       1.151210
florida       3.184256
oregon        0.740273
nevada        0.021147
california   -0.697979
idaho         0.021147
dtype: float64

In [66]:
fill_values = {'East': 0.5, 'West': -1}

def fill_function(group: DataFrameGroupBy):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_function)

C:\Users\William Costa\AppData\Local\Temp\ipykernel_8988\3849310054.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_function)


ohio         -0.504705
ny            0.774080
vermont       0.500000
florida       3.184256
oregon        0.740273
nevada       -1.000000
california   -0.697979
idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [68]:
suits = ['h', 's', 'c', 'd']
card_val = (list(range(1,11)) + [10] * 3) * 4 
base_names = ['a'] + list(range(2,11)) + ['j', 'k', 'q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

deck

ah      1
2h      2
3h      3
4h      4
5h      5
6h      6
7h      7
8h      8
9h      9
10h    10
jh     10
kh     10
qh     10
as      1
2s      2
3s      3
4s      4
5s      5
6s      6
7s      7
8s      8
9s      9
10s    10
js     10
ks     10
qs     10
ac      1
2c      2
3c      3
4c      4
5c      5
6c      6
7c      7
8c      8
9c      9
10c    10
jc     10
kc     10
qc     10
ad      1
2d      2
3d      3
4d      4
5d      5
6d      6
7d      7
8d      8
9d      9
10d    10
jd     10
kd     10
qd     10
dtype: int64

In [69]:
def draw(deck: pd.Series, n=5):
    return deck.sample(n)

draw(deck)

5s     5
7h     7
3s     3
qh    10
6d     6
dtype: int64

In [71]:
def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2), deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

(c  5c     5
    7c     7
 d  7d     7
    kd    10
 h  3h     3
    qh    10
 s  3s     3
    ks    10
 dtype: int64,
 6c      6
 10c    10
 5d      5
 9d      9
 7h      7
 6h      6
 js     10
 10s    10
 dtype: int64)

### Example: Group Weighted Average and Correlation

In [74]:
frame = pd.DataFrame({
    'category': list('aaaabbbb'),
    'data': np.random.standard_normal(8),
    'weights': np.random.uniform(size=8),
})
frame

,category,data,weights
0,a,0.086372,0.192192
1,a,-1.319402,0.799483
2,a,0.797410,0.234450
3,a,0.916538,0.320794
4,b,-1.265675,0.082628
5,b,0.543128,0.620932
6,b,1.348917,0.424457
7,b,-0.541860,0.109240


In [75]:
grouped = frame.groupby('category')

def get_weighted_average(group: DataFrameGroupBy):
    return np.average(group['data'], weights=group['weights'])


grouped.apply(get_weighted_average)

category
a   -0.360242
b    0.602971
dtype: float64

In [76]:
location = r"C:\Users\William Costa\Documents\repositories\data_science_basic_knowledge\resources\stock_px.csv"

close_px = pd.read_csv(location, parse_dates=True, index_col=0)

close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [77]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [80]:
def spx_corr(group: DataFrameGroupBy):
    return group.corrwith(group['SPX'])

rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [81]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [82]:
def corr_aapl_msft(group: DataFrameGroupBy):
    return group['AAPL'].corr(group['MSFT'])

by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [85]:
!pip install statsmodels

     ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
     --- ------------------------------------ 0.7/9.1 MB 22.7 MB/s eta 0:00:01
     --------- ------------------------------ 2.3/9.1 MB 28.4 MB/s eta 0:00:01
     ---------------------- ----------------- 5.1/9.1 MB 40.6 MB/s eta 0:00:01
     ------------------------------------- -- 8.5/9.1 MB 49.5 MB/s eta 0:00:01
     ---------------------------------------- 9.1/9.1 MB 44.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/233.8 kB ? eta -:--:--
     ---------------------------------------- 233.8/233.8 kB ? eta 0:00:00


In [86]:
import statsmodels.api as sm

def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, yvar='AAPL', xvars=['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Group Transforms and "Unwrapped" GroupBys

In [88]:
frame = pd.DataFrame({
    'key': list('abc')*4, 'value': np.arange(12.)
})
frame

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [90]:
g = frame.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [91]:
def get_mean(group: DataFrameGroupBy):
    return group.mean()

g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [92]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [93]:
def times_two(group: DataFrameGroupBy):
    return group * 2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [94]:
def get_ranks(group: DataFrameGroupBy):
    return group.rank(ascending=False)

g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [95]:
def normalize(group: DataFrameGroupBy):
    return (group - group.mean()) / group.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [97]:
normalized = (frame['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## Pivot Tables and Cross-Tabulation

In [98]:
tips.pivot_table(index=['day', 'smoker'])

C:\Users\William Costa\AppData\Local\Temp\ipykernel_8988\2966571272.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=['day', 'smoker'])


size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [101]:
tips.pivot_table(index=['time', 'day'], columns='smoker', values=['tips_pct', 'size'])

size            tips_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [102]:
tips.pivot_table(index=['time', 'day'], columns='smoker',
                 values=['tips_pct', 'size'], margins=True)

size                      tips_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [103]:
tips.pivot_table(index=['time', 'smoker'], columns='day',
                 values='tips_pct', margins=True, aggfunc=len)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [104]:
tips.pivot_table(index=['time', 'smoker'], columns='day', fill_value=0,
                 values='tips_pct', margins=True, aggfunc=len)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

## Cross-Tabulations: Crosstab

In [106]:
from io import StringIO

data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

data = pd.read_table(StringIO(data), sep='\s+')
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [107]:
pd.crosstab(data['Nationality'], data['Handedness'], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [109]:
pd.crosstab([tips['time'], tips['day']], tips['smoker'], margins=True)


smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244